In [2]:
import pandas as pd
import numpy as np

In [ ]:
# อ่านข้อมูล metadata ของแต่ละ site
site_df = pd.read_csv('../data/DeDe_49stations/DeDe_49sites_metadata.csv')
for i, row in site_df.iterrows():
    row = site_df.iloc[-1,:]
    # อ่าน raw data ที่ดาวน์โหลดมา
    df_15min = pd.read_csv(f'../data/CAMS/cams_time_series_15min/raw_data/cams_solar_rad_alti_8site{row['Site_id']}.csv',skiprows=42,sep=';')
    df_1hr = pd.read_csv(f'../data/CAMS/cams_time_series_1hr/raw_data/cams_solar_rad_period_1hr_site{row['Site_id']}.csv',skiprows=42,sep=';')

    # เปลี่ยนชื่อ columns เพื่อนำไปใช้ต่อไป
    df_15min['Datetime'] = df_15min['# Observation period'].apply(lambda x : x.split('/')[-1])
    df_15min['Datetime'] = pd.to_datetime(df_15min['Datetime'])
    df_15min.drop(columns=['# Observation period'], inplace=True)

    df_1hr['Datetime'] = df_1hr['# Observation period'].apply(lambda x : x.split('/')[-1])
    df_1hr['Datetime'] = pd.to_datetime(df_1hr['Datetime'])
    df_1hr.drop(columns=['# Observation period'], inplace=True)

    # ปรับ timezone จาก UTC+0 ให้เป็น UTC+7
    df_15min.set_index('Datetime', inplace=True)
    df_15min = df_15min.tz_localize('UTC').tz_convert('Asia/Bangkok').tz_localize(None)
    df_15min.reset_index(inplace=True)

    df_1hr.set_index('Datetime', inplace=True)
    df_1hr = df_1hr.tz_localize('UTC').tz_convert('Asia/Bangkok').tz_localize(None)
    df_1hr.reset_index(inplace=True)

    # กรองเฉพาะข้อมูลที่อยู่ในปี 2023
    df_15min = df_15min[df_15min['Datetime'].dt.year == 2023]
    df_1hr = df_1hr[df_1hr['Datetime'].dt.year == 2023]

    # เก็บข้อมูล
    df_15min.to_csv(f'../data/CAMS/cams_time_series_15min/cams_time_series_15min_site{row['Site_id']:02d}.csv',index=False)
    df_1hr.to_csv(f'../data/CAMS/cams_time_series_1hr/cams_time_series_1hr_site{row['Site_id']:02d}.csv',index=False)